<a href="https://colab.research.google.com/github/mlaricobar/solution-datathon-interbank-2020/blob/desarrollo/%5B03%5D%20Feature%20Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

In [1]:
from google.colab import drive, files

import pandas as pd

from datetime import datetime
from dateutil import relativedelta

In [2]:
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competencias/'[02] Interbank-2020'
!ls -lh '[00] Data'

/gdrive/My Drive/Competencias/[02] Interbank-2020
total 5.1G
-rw------- 1 root root 979M Dec 12 01:33 censo_test.csv
-rw------- 1 root root 748M Dec 12 01:34 censo_train.csv
-rw------- 1 root root  998 Dec 12 01:34 productos.csv
-rw------- 1 root root 1.7G Dec 12 01:34 rcc_test.csv
-rw------- 1 root root 1.4G Dec 12 01:35 rcc_train.csv
-rw------- 1 root root 3.3M Dec 12 01:35 sample_submission.csv
-rw------- 1 root root  30M Dec 12 01:35 se_test.csv
-rw------- 1 root root  28M Dec 12 01:35 se_train.csv
-rw------- 1 root root 122M Dec 12 01:35 sunat_test.csv
-rw------- 1 root root 106M Dec 12 01:35 sunat_train.csv
-rw------- 1 root root 3.0M Dec 12 01:35 y_train.csv


## Dataset RCC

In [4]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [20]:
df_rcc_train = optimizar_df(pd.read_csv("[00] Data/rcc_train.csv"))
df_rcc_test = optimizar_df(pd.read_csv("[00] Data/rcc_test.csv"))

In [6]:
df_rcc_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30391626 entries, 0 to 30391625
Data columns (total 9 columns):
 #   Column                    Non-Null Count     Dtype  
---  ------                    --------------     -----  
 0   codmes                    30391626 non-null  uint32 
 1   key_value                 30391626 non-null  uint32 
 2   condicion                 30391626 non-null  uint16 
 3   tipo_credito              30391626 non-null  uint8  
 4   cod_instit_financiera     30391626 non-null  uint8  
 5   saldo                     30391626 non-null  float32
 6   PRODUCTO                  30391626 non-null  uint8  
 7   RIESGO_DIRECTO            30391626 non-null  int64  
 8   COD_CLASIFICACION_DEUDOR  30391626 non-null  uint8  
dtypes: float32(1), int64(1), uint16(1), uint32(2), uint8(4)
memory usage: 753.6 MB


In [7]:
df_rcc_train.head()

,codmes,key_value,condicion,tipo_credito,cod_instit_financiera,saldo,PRODUCTO,RIESGO_DIRECTO,COD_CLASIFICACION_DEUDOR
0,201711,4,0,12,33,-0.072601,6,-1,0
1,201711,4,0,12,33,-0.070608,7,-1,0
2,201711,4,0,12,61,-0.052054,5,-1,0
3,201711,4,0,11,32,-0.072456,8,1,0
4,201711,4,0,12,61,-0.057877,0,1,0


In [21]:
top_freq_products = [1, 2, 3, 8, 4, 0, 6]
top_propensity_products = [12, 17, 18, 25, 20]
low_propensity_products = [32, 33, 35, 36, 37, 38, 40, 41]
list_product = top_freq_products + top_propensity_products + low_propensity_products
print(list_product)

[1, 2, 3, 8, 4, 0, 6, 12, 17, 18, 25, 20, 32, 33, 35, 36, 37, 38, 40, 41]


In [22]:
cat_col = "PRODUCTO"
df_rcc_train.loc[~df_rcc_train[cat_col].isin(list_product), cat_col] = "OTROS"
df_rcc_train.loc[df_rcc_train[cat_col].isin(top_propensity_products), cat_col] = "TOP_PROPENSITY"
df_rcc_train.loc[df_rcc_train[cat_col].isin(low_propensity_products), cat_col] = "LOW_PROPENSITY"

In [24]:
agg_func = "mean"
numeric_value = "saldo"
df_rcc_crosstab = (pd.crosstab(index=[df_rcc_train.key_value, df_rcc_train.codmes], 
                               columns=df_rcc_train[cat_col], 
                               values=df_rcc_train[numeric_value],
                               aggfunc=agg_func)
                    .reset_index(drop=False)
                    .reset_index(drop=True))
df_rcc_crosstab.columns = ["{0}_{1}_FOR_{2}_{3}".format(numeric_value.upper(), agg_func.upper(), cat_col, c) if c not in ["key_value", "codmes"] else c for c in df_rcc_crosstab.columns]

feat_cols = [c for c in df_rcc_crosstab.columns if "{0}_".format(cat_col) in c]
df_rcc_crosstab["MEAN_OF_{0}_{1}_FOR_ALL_{2}".format(numeric_value.upper(), agg_func.upper(), cat_col)] = df_rcc_crosstab[feat_cols].mean(axis=1)
df_rcc_crosstab["SUM_OF_{0}_{1}_FOR_ALL_{2}".format(numeric_value.upper(), agg_func.upper(), cat_col)] = df_rcc_crosstab[feat_cols].sum(axis=1)
df_rcc_crosstab["COUNT_OF_{0}_{1}_FOR_ALL_{2}".format(numeric_value.upper(), agg_func.upper(), cat_col)] = df_rcc_crosstab[feat_cols].count(axis=1)

In [25]:
df_rcc_crosstab.head(10)

,key_value,codmes,SALDO_MEAN_FOR_PRODUCTO_0,SALDO_MEAN_FOR_PRODUCTO_1,SALDO_MEAN_FOR_PRODUCTO_2,SALDO_MEAN_FOR_PRODUCTO_3,SALDO_MEAN_FOR_PRODUCTO_4,SALDO_MEAN_FOR_PRODUCTO_6,SALDO_MEAN_FOR_PRODUCTO_8,SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY,SALDO_MEAN_FOR_PRODUCTO_OTROS,SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY,MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO,SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO,COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO
0,0,201703,NaN,-0.072814,-0.038181,-0.041831,NaN,NaN,-0.069253,NaN,NaN,NaN,-0.055520,-0.222079,4
1,0,201704,NaN,-0.072556,-0.038181,-0.044653,NaN,NaN,-0.066431,NaN,-0.072814,NaN,-0.058927,-0.294635,5
2,0,201705,NaN,-0.072713,-0.038181,-0.043309,NaN,NaN,-0.067775,NaN,-0.072814,NaN,-0.058958,-0.294792,5
3,0,201706,NaN,-0.072691,-0.038181,-0.044518,NaN,NaN,-0.066566,NaN,-0.072814,NaN,-0.058954,-0.294769,5
4,0,201707,NaN,-0.072388,-0.038181,-0.049154,NaN,NaN,-0.061930,NaN,-0.072814,NaN,-0.058893,-0.294467,5
5,0,201708,NaN,-0.072265,-0.038181,-0.047889,NaN,NaN,-0.063195,NaN,-0.072814,NaN,-0.058869,-0.294344,5
6,0,201709,0.072673,-0.072052,-0.038181,-0.038181,NaN,NaN,NaN,NaN,NaN,NaN,-0.018935,-0.075741,4
7,0,201710,0.070836,NaN,-0.038181,-0.038181,NaN,NaN,NaN,NaN,NaN,NaN,-0.001842,-0.005525,3
8,0,201711,0.070836,-0.071649,-0.038181,-0.038181,NaN,NaN,NaN,NaN,NaN,NaN,-0.019294,-0.077174,4
9,0,201712,0.064219,NaN,-0.038181,-0.038181,NaN,NaN,NaN,NaN,NaN,NaN,-0.004048,-0.012143,3


In [26]:
month_dict = {"U12M": [201802, 201801, 201712, 201711, 201710, 201709, 201708, 201707, 201706, 201705, 201704, 201703],
              "U06M": [201802, 201801, 201712, 201711, 201710, 201709],
              "U03M": [201802, 201801, 201712],
              "U01M": [201802]
              }

In [28]:
def diff_months(codmes_ini, codmes_fin):
  r = relativedelta.relativedelta(datetime.strptime(str(int(codmes_fin)), '%Y%m'), datetime.strptime(str(int(codmes_ini)), '%Y%m'))
  return r.years*12 + r.months

In [27]:
agg_col_dict = {"codmes": ["last", "first"]}
agg_col_dict.update({c: ["mean", "sum", "count"] for c in df_rcc_crosstab.columns if c not in ["key_value", "codmes"]})

{'COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO': ['mean', 'sum', 'count'],
 'MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_0': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_1': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_2': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_3': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_4': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_6': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_8': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_OTROS': ['mean', 'sum', 'count'],
 'SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY': ['mean', 'sum', 'count'],
 'SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO': ['mean', 'sum', 'count'],
 'codmes': ['last', 'first']}

In [29]:
month_level = "U03M"
df_rcc_agg = (df_rcc_crosstab.loc[df_rcc_crosstab["codmes"].isin(month_dict[month_level])]
                             .groupby(["key_value"], as_index=False)
                             .agg(agg_col_dict)
                )
df_rcc_agg.columns = [month_level + "_" + "_".join(c).upper() if c[-1] != "" else c[0] for c in df_rcc_agg.columns]

In [30]:
df_rcc_agg["{0}_RECENCY".format(month_level)] = df_rcc_agg.apply(lambda row: diff_months(row["{0}_CODMES_LAST".format(month_level)], 201803), axis=1)
df_rcc_agg["{0}_LONG_RECENCY".format(month_level)] = df_rcc_agg.apply(lambda row: diff_months(row["{0}_CODMES_FIRST".format(month_level)], 201803), axis=1)
df_rcc_agg["{0}_DIFF_MONTHS_FIRST_LAST".format(month_level)] = df_rcc_agg.apply(lambda row: diff_months(row["{0}_CODMES_FIRST".format(month_level)], row["{0}_CODMES_LAST".format(month_level)]), axis=1)
del df_rcc_agg["{0}_CODMES_FIRST".format(month_level)]
del df_rcc_agg["{0}_CODMES_LAST".format(month_level)]

In [35]:
df_rcc_agg.head()

,key_value,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_LOW_PROPENSITY_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_OTROS_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_SUM,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_COUNT,U03M_RECENCY,U03M_LONG_RECENCY,U03M_DIFF_MONTHS_FIRST_LAST
0,0,0.062248,0.186743,3,-0.072355,-0.072355,1,-0.038181,-0.114542,3,-0.038442,-0.115326,3,NaN,0.000000,0,NaN,0.000000,0,-0.072120,-0.072120,1,NaN,0.0,0,NaN,0.000000,0,NaN,0.0,0,-0.013735,-0.041204,3,-0.062533,-0.187599,3,3.666667,11,3,1,3,2
1,1,NaN,0.000000,0,-0.072717,-0.218150,3,NaN,0.000000,0,NaN,0.000000,0,-0.062238,-0.186714,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,-0.072881,-0.072881,1,NaN,0.0,0,-0.068062,-0.204186,3,-0.159249,-0.477746,3,2.333333,7,3,1,3,2
2,2,NaN,0.000000,0,-0.072758,-0.072758,1,NaN,0.000000,0,NaN,0.000000,0,-0.004746,-0.004746,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,-0.072881,-0.072881,1,NaN,0.0,0,-0.050128,-0.050128,1,-0.150384,-0.150384,1,3.000000,3,1,1,1,0
3,3,NaN,0.000000,0,-0.072791,-0.072791,1,NaN,0.000000,0,NaN,0.000000,0,0.095067,0.095067,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,0.095067,0.095067,1,NaN,0.0,0,0.039114,0.039114,1,0.117343,0.117343,1,3.000000,3,1,1,1,0
4,4,-0.056247,-0.168741,3,-0.072403,-0.217210,3,-0.058336,-0.175007,3,-0.067921,-0.203762,3,NaN,0.000000,0,-0.070507,-0.211522,3,-0.063319,-0.189956,3,NaN,0.0,0,-0.061331,-0.183993,3,NaN,0.0,0,-0.064295,-0.192884,3,-0.450063,-1.350190,3,7.000000,21,3,1,3,2


In [37]:
df_rcc_agg.to_csv("[01] Intermediate/ds_rcc.csv", index=False)

In [38]:
!ls -l

total 99144
drwx------ 2 root root     4096 Dec 12 01:33 '[00] Data'
drwx------ 2 root root     4096 Dec 17 12:43 '[01] Intermediate'
-rw------- 1 root root 84685747 Dec 17 12:43  ds_rcc
-rw------- 1 root root  8414286 Dec 12 02:54  pfe_se_test.html
-rw------- 1 root root  8414287 Dec 12 02:26  pfe_se_train.html
